In [34]:
from pathlib import Path
import joblib
import numpy as np
import pandas as pd

from sklearn.calibration import CalibratedClassifierCV
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import brier_score_loss, log_loss
from sklearn.model_selection import GroupShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder


# DNF model
Train a logistic regression hazard model from lap-level data and use it to simulate retirements per lap.
Also exposes summary DNF rates per circuit/year for inspection.


In [35]:
def load_default_df():
    candidates = [
        Path("driver_lap_dataset.csv"),
        Path("models/driver_lap_dataset.csv"),
        Path("fastf1_lap_dataset.csv"),
        Path("models/fastf1_lap_dataset.csv"),
    ]
    csv_path = next((p for p in candidates if p.exists()), None)
    if csv_path is None:
        raise FileNotFoundError(
            "No lap dataset found (driver_lap_dataset.csv / fastf1_lap_dataset.csv)."
        )
    df = pd.read_csv(csv_path)
    if "dnf" not in df.columns:
        df["dnf"] = False
    return df

df = load_default_df()
df.head()


,driver_id,team_id,team_name,circuit_id,total_race_laps,year,session_key,race_name,session_name,grid_position,...,track_temperature,air_temperature,has_rain,humidity,pressure,rainfall,wind_speed,wind_direction,is_pit,dnf
0,703,48,Mercedes,yas_marina,55.0,2012,2012_abu_dhabi_grand_prix_r,Abu Dhabi Grand Prix,Abu Dhabi Grand Prix R,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,703,48,Mercedes,yas_marina,55.0,2012,2012_abu_dhabi_grand_prix_r,Abu Dhabi Grand Prix,Abu Dhabi Grand Prix R,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,703,48,Mercedes,yas_marina,55.0,2012,2012_abu_dhabi_grand_prix_r,Abu Dhabi Grand Prix,Abu Dhabi Grand Prix R,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,703,48,Mercedes,yas_marina,55.0,2012,2012_abu_dhabi_grand_prix_r,Abu Dhabi Grand Prix,Abu Dhabi Grand Prix R,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,703,48,Mercedes,yas_marina,55.0,2012,2012_abu_dhabi_grand_prix_r,Abu Dhabi Grand Prix,Abu Dhabi Grand Prix R,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [36]:
# Training
include_year = True
min_laps_prior = 500.0

df_sorted = df.sort_values(["session_key", "driver_id", "lap_number"])
prev_dnf = df_sorted.groupby(["session_key", "driver_id"])["dnf"].shift(fill_value=False)
df_sorted["dnf_event"] = df_sorted["dnf"] & ~prev_dnf
event_seen = df_sorted.groupby(["session_key", "driver_id"])["dnf_event"].cumsum()
train_df = df_sorted[(event_seen == 0) | df_sorted["dnf_event"]]

avg_total_laps = float(train_df["total_race_laps"].dropna().mean() or 1.0)

def build_features(frame, avg_total_laps, include_year):
    circuit = frame["circuit_id"].astype(str).fillna("unknown")
    lap_number = frame["lap_number"].fillna(0).astype(float)
    total_laps = frame["total_race_laps"].fillna(avg_total_laps).astype(float)
    progress = lap_number / total_laps.replace(0, np.nan)
    progress = progress.fillna(0)

    data = {
        "circuit_id": circuit,
        "lap_number": lap_number,
        "total_race_laps": total_laps,
        "progress": progress,
    }
    if include_year:
        data["year"] = frame["year"].apply(
            lambda v: str(int(v)) if pd.notna(v) else "unknown"
        )
    return pd.DataFrame(data)

X = build_features(train_df, avg_total_laps, include_year)
y = train_df["dnf_event"].astype(int).to_numpy()
groups = train_df["session_key"].astype(str).fillna("unknown")

categorical = ["circuit_id"]
if include_year:
    categorical.append("year")
numeric = ["lap_number", "total_race_laps", "progress"]

preprocess = ColumnTransformer(
    [
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
        ("num", "passthrough", numeric),
    ]
)
base_pipeline = Pipeline([
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=2000, class_weight="balanced")),
])

gss_outer = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, temp_idx = next(gss_outer.split(X, y, groups=groups))
X_train, y_train = X.iloc[train_idx], y[train_idx]
X_temp, y_temp = X.iloc[temp_idx], y[temp_idx]
groups_temp = groups.iloc[temp_idx]

gss_inner = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=43)
cal_idx, test_idx = next(gss_inner.split(X_temp, y_temp, groups=groups_temp))
X_cal, y_cal = X_temp.iloc[cal_idx], y_temp[cal_idx]
X_test, y_test = X_temp.iloc[test_idx], y_temp[test_idx]

base_pipeline.fit(X_train, y_train)
calibrator = CalibratedClassifierCV(base_pipeline, method="sigmoid", cv="prefit")
calibrator.fit(X_cal, y_cal)
pipeline = calibrator

def calibration_error(y_true, y_prob, n_bins=10):
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    bin_ids = np.digitize(y_prob, bins) - 1
    ece = 0.0
    for b in range(n_bins):
        mask = bin_ids == b
        if not np.any(mask):
            continue
        bin_acc = y_true[mask].mean()
        bin_conf = y_prob[mask].mean()
        ece += abs(bin_acc - bin_conf) * (mask.mean())
    return ece

test_probs = pipeline.predict_proba(X_test)[:, 1]
test_log_loss = log_loss(y_test, test_probs)
test_brier = brier_score_loss(y_test, test_probs)
test_ece = calibration_error(y_test, test_probs, n_bins=10)
print(f"Test log loss: {test_log_loss:.6f}")
print(f"Test Brier score: {test_brier:.6f}")
print(f"Test ECE: {test_ece:.6f}")

runs = (
    df.groupby(["session_key", "driver_id", "circuit_id", "total_race_laps", "year", "race_name"])
    .agg(last_lap=("lap_number", "max"), dnf=("dnf", "max"))
    .reset_index()
)
runs["dnf"] = runs["dnf"].fillna(False).astype(bool)
runs["exposure_laps"] = runs["last_lap"].astype(float)
total_events = float(runs["dnf"].sum())
total_laps = float(runs["exposure_laps"].sum())
global_hazard = (total_events / total_laps) if total_laps > 0 else 0.0

def aggregate_hazard(runs, group_cols, global_hazard, min_laps_prior):
    events = runs.groupby(group_cols)["dnf"].sum()
    exposure = runs.groupby(group_cols)["exposure_laps"].sum()
    stats = pd.concat([events, exposure], axis=1).rename(
        columns={"dnf": "dnfs", "exposure_laps": "exposure_laps"}
    )
    prior_events = global_hazard * min_laps_prior
    stats["hazard"] = (stats["dnfs"] + prior_events) / (stats["exposure_laps"] + min_laps_prior)
    stats["hazard"] = stats["hazard"].clip(lower=1e-4, upper=0.5)
    return stats

dnf_stats = aggregate_hazard(runs, ["circuit_id"], global_hazard, min_laps_prior)
dnf_stats_by_year = aggregate_hazard(runs, ["year"], global_hazard, min_laps_prior) if include_year else None

# Export
model_path = Path("models/dnf_model.joblib")
model_path.parent.mkdir(parents=True, exist_ok=True)
bundle = {
    "pipeline": pipeline,
    "include_year": include_year,
    "avg_total_laps": avg_total_laps,
    "global_hazard": global_hazard,
}
joblib.dump(bundle, model_path)


Test log loss: 0.004018
Test Brier score: 0.000776
Test ECE: 0.000214


/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/sklearn/calibration.py:330: FutureWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


['models/dnf_model.joblib']

In [27]:
from IPython.display import display

print(f"Global hazard per lap: {global_hazard:.5f}")

print("\nTop circuits by DNF hazard:")
display(dnf_stats.sort_values("hazard", ascending=False).head(10))

if dnf_stats_by_year is not None:
    print("\nTop years by DNF hazard:")
    display(dnf_stats_by_year.sort_values("hazard", ascending=False).head(10))


Global hazard per lap: 0.00076

Top circuits by DNF hazard:


,dnfs,exposure_laps,hazard
circuit_id,,,
mugello,4,778.0,0.003428
valencia,3,1210.0,0.001977
yeongam,5,2276.0,0.001939
jeddah,7,4396.0,0.001508
monaco,23,18194.0,0.001251
istanbul,3,2223.0,0.001242
americas,15,13308.0,0.001114
buddh,3,2569.0,0.001102
marina_bay,15,13507.0,0.001098



Top years by DNF hazard:


,dnfs,exposure_laps,hazard
year,,,
2012,32,25343.0,0.001253
2019,26,23625.0,0.001094
2022,24,23529.0,0.001015
2013,22,22779.0,0.000961
2020,17,18400.0,0.000920
2018,20,22339.0,0.000892
2017,18,20307.0,0.000883
2021,20,23688.0,0.000843
2014,17,21053.0,0.000806


# Model inference

Once model is trained, code below can help to run smoke test

### Inference over synthetic data 

This will generate 70 laps for all circuit for all years (so the distribution of laps can be off)

In [ ]:
# Sampled hazards per circuit/year using the trained model
from tqdm.auto import tqdm

rng = np.random.default_rng(12325)
circuits = df['circuit_id'].dropna().unique().tolist()
years = sorted(df['year'].dropna().unique().tolist())

median_total_laps = (
    df.groupby(['circuit_id', 'year'])['total_race_laps']
    .median()
    .to_dict()
)

def dnf_hazard(circuit_id, lap_number, year=None, total_race_laps=None):
    total_laps = float(total_race_laps) if total_race_laps else float(avg_total_laps or 1.0)
    lap_num = float(lap_number or 0)
    progress = lap_num / total_laps if total_laps > 0 else 0.0

    data = {
        'circuit_id': str(circuit_id) if circuit_id is not None else 'unknown',
        'lap_number': lap_num,
        'total_race_laps': total_laps,
        'progress': progress,
    }
    if include_year:
        data['year'] = str(year) if year is not None else 'unknown'
    X = pd.DataFrame([data])
    prob = float(pipeline.predict_proba(X)[0, 1])
    return float(np.clip(prob, 1e-6, 0.5))

rows = []
combos = [(circuit_id, year) for circuit_id in circuits for year in years]
for circuit_id, year in tqdm(combos, desc='Sampling circuit/year'):
    total_laps = median_total_laps.get((circuit_id, year), avg_total_laps)
    if not total_laps or total_laps <= 0:
        total_laps = avg_total_laps
    total_laps = int(round(total_laps))
    lap_samples = rng.integers(1, total_laps + 1, size=75)
    for lap_number in lap_samples:
        rows.append({
            'circuit_id': circuit_id,
            'year': year,
            'lap_number': int(lap_number),
            'total_race_laps': total_laps,
            'hazard': dnf_hazard(circuit_id, lap_number, year=year, total_race_laps=total_laps),
        })

sampled = pd.DataFrame(rows)
global_hazard_sampled = float(sampled['hazard'].mean())
print(f"Global hazard per lap: {global_hazard_sampled:.5f}")
sampled_summary = (
    sampled.groupby(['circuit_id', 'year'])['hazard']
    .mean()
    .reset_index()
    .sort_values('hazard', ascending=False)
)
display(sampled_summary.head(20))

print('\nTop circuits by DNF hazard (sampled):')
sampled_circuits = (
    sampled.groupby('circuit_id')['hazard']
    .mean()
    .sort_values(ascending=False)
)
display(sampled_circuits.head(10))

print('\nTop years by DNF hazard (sampled):')
sampled_years = (
    sampled.groupby('year')['hazard']
    .mean()
    .sort_values(ascending=False)
)
display(sampled_years.head(10))


Sampling circuit/year: 100%|██████████| 490/490 [00:39<00:00, 12.51it/s]


Global hazard per lap: 0.00058


,circuit_id,year,hazard
387,spa,2021,0.010080
21,americas,2019,0.001500
470,yeongam,2020,0.001368
301,ricard,2019,0.001309
310,rodriguez,2014,0.001262
265,nurburgring,2025,0.001255
34,bahrain,2018,0.001237
402,suzuka,2022,0.001222
354,silverstone,2016,0.001196
478,zandvoort,2014,0.001167



Top circuits by DNF hazard (sampled):


circuit_id
spa               0.001237
baku              0.000738
silverstone       0.000683
mugello           0.000666
americas          0.000657
bahrain           0.000648
shanghai          0.000637
albert_park       0.000627
hockenheimring    0.000623
ricard            0.000622
Name: hazard, dtype: float64


Top years by DNF hazard (sampled):


year
2021    0.000861
2019    0.000672
2014    0.000660
2020    0.000623
2016    0.000587
2022    0.000559
2012    0.000552
2025    0.000547
2013    0.000543
2018    0.000526
Name: hazard, dtype: float64

## Inference over the entire dataset

In [30]:
# Predicted hazards over the full lap dataset
full_features = build_features(df, avg_total_laps, include_year)
full_probs = pipeline.predict_proba(full_features)[:, 1]
pred_df = df[["circuit_id", "year"]].copy()
pred_df["hazard"] = full_probs

global_hazard_pred = float(pred_df["hazard"].mean())
print(f"Global hazard per lap: {global_hazard_pred:.5f}")

pred_summary = (
    pred_df.groupby(["circuit_id", "year"])["hazard"]
    .mean()
    .reset_index()
    .sort_values("hazard", ascending=False)
)
display(pred_summary.head(20))

print('\nTop circuits by DNF hazard (predicted):')
pred_circuits = (
    pred_df.groupby("circuit_id")["hazard"]
    .mean()
    .sort_values(ascending=False)
)
display(pred_circuits.head(10))

print('\nTop years by DNF hazard (predicted):')
pred_years = (
    pred_df.groupby("year")["hazard"]
    .mean()
    .sort_values(ascending=False)
)
display(pred_years.head(10))


Global hazard per lap: 0.00059


,circuit_id,year,hazard
232,spa,2021,0.010080
245,suzuka,2022,0.000974
2,albert_park,2014,0.000944
225,spa,2014,0.000935
231,spa,2020,0.000889
223,spa,2012,0.000881
6,albert_park,2018,0.000869
5,albert_park,2017,0.000867
228,spa,2017,0.000865
224,spa,2013,0.000865



Top circuits by DNF hazard (predicted):


circuit_id
spa            0.000848
mugello        0.000810
albert_park    0.000798
baku           0.000703
silverstone    0.000696
jeddah         0.000692
americas       0.000670
monza          0.000663
valencia       0.000663
ricard         0.000663
Name: hazard, dtype: float64


Top years by DNF hazard (predicted):


year
2014    0.000688
2012    0.000636
2022    0.000619
2017    0.000617
2019    0.000611
2013    0.000609
2020    0.000608
2021    0.000603
2018    0.000599
2016    0.000584
Name: hazard, dtype: float64

# Bias analysis

In [33]:
# Compare model vs empirical per circuit
full_features = build_features(df, avg_total_laps, include_year)
full_probs = pipeline.predict_proba(full_features)[:, 1]
pred_df = df[["circuit_id", "year"]].copy()
pred_df["hazard"] = full_probs
pred_circuit = pred_df.groupby("circuit_id")["hazard"].mean()

runs = (
    df.groupby(["session_key", "driver_id", "circuit_id", "total_race_laps", "year", "race_name"])
    .agg(last_lap=("lap_number", "max"), dnf=("dnf", "max"))
    .reset_index()
)
runs["dnf"] = runs["dnf"].fillna(False).astype(bool)
runs["exposure_laps"] = runs["last_lap"].astype(float)
emp = (
    runs.groupby("circuit_id")
    .agg(dnfs=("dnf", "sum"), exposure_laps=("exposure_laps", "sum"))
)
emp["emp_hazard"] = emp["dnfs"] / emp["exposure_laps"]

compare = pred_circuit.to_frame("pred_hazard").join(emp[["emp_hazard"]], how="inner")
compare["pred_minus_emp"] = compare["pred_hazard"] - compare["emp_hazard"]
compare["abs_error"] = compare["pred_minus_emp"].abs()

print('Largest positive prediction bias (top 10):')
display(compare.sort_values("pred_minus_emp", ascending=False).head(10))

print('Largest negative prediction bias (top 10):')
display(compare.sort_values("pred_minus_emp", ascending=True).head(10))

print('Largest absolute error (top 10):')
display(compare.sort_values("abs_error", ascending=False).head(10))


Largest positive prediction bias (top 10):


,pred_hazard,emp_hazard,pred_minus_emp,abs_error
circuit_id,,,,
vegas,0.000552,0.000000,0.000552,0.000552
losail,0.000538,0.000000,0.000538,0.000538
zandvoort,0.000486,0.000291,0.000196,0.000196
bahrain,0.000601,0.000422,0.000179,0.000179
red_bull_ring,0.000481,0.000390,0.000091,0.000091
sochi,0.000622,0.000533,0.000088,0.000088
hungaroring,0.000455,0.000377,0.000078,0.000078
yas_marina,0.000590,0.000512,0.000078,0.000078
miami,0.000538,0.000465,0.000073,0.000073


Largest negative prediction bias (top 10):


,pred_hazard,emp_hazard,pred_minus_emp,abs_error
circuit_id,,,,
mugello,0.000810,0.005141,-0.004332,0.004332
valencia,0.000663,0.002479,-0.001817,0.001817
yeongam,0.000653,0.002197,-0.001544,0.001544
jeddah,0.000692,0.001592,-0.000900,0.000900
monaco,0.000493,0.001264,-0.000772,0.000772
istanbul,0.000627,0.001350,-0.000723,0.000723
buddh,0.000602,0.001168,-0.000566,0.000566
marina_bay,0.000604,0.001111,-0.000507,0.000507
americas,0.000670,0.001127,-0.000457,0.000457


Largest absolute error (top 10):


,pred_hazard,emp_hazard,pred_minus_emp,abs_error
circuit_id,,,,
mugello,0.000810,0.005141,-0.004332,0.004332
valencia,0.000663,0.002479,-0.001817,0.001817
yeongam,0.000653,0.002197,-0.001544,0.001544
jeddah,0.000692,0.001592,-0.000900,0.000900
monaco,0.000493,0.001264,-0.000772,0.000772
istanbul,0.000627,0.001350,-0.000723,0.000723
buddh,0.000602,0.001168,-0.000566,0.000566
vegas,0.000552,0.000000,0.000552,0.000552
losail,0.000538,0.000000,0.000538,0.000538
